# Tools
## Motivation
LLM的局限性：
- 知识戒指日期
- 计算能力限制与代码执行限制
- 无法与外部世界交互：访问网页、调用API等

目前工具的实现解决了：
- 获取实时数据：通过调用天气API、搜索引擎API等
- 执行计算或代码：利用计算工具进行复杂的数学运算、数据处理等
- 操作外部系统：读写文件、与数据哭交互、调用其他应用程序的API
- 利用专业能力：翻译、图像生成

没有Tools，Agent只是一个文本信息反馈的模型，加上Tools，Agent变成了能处理某些业务的Agent

## Definition
`openai-agents`框架中的三类工具：
1. 托管工具：（Hosted Tools）：
    - 工具集成在模型服务器端运行。
    - 直接由特定LLM提供商提供，调用API使用
2. 函数调用（Function Calling）：
    - 使用任何Pyhton函数包装成Agent可以调用的工具
    - 极其灵活，可自定义任何功能
    - Agent通过函数签名和文档字符串（docstring）来理解函数的参数和返回值（理解如何使用工具）
3. Agent作为工具（Agent as a Tool）：
    - 允许编排者Agent在其内部循环中调用其他专家Agent
    - 实现了Agent之间的合作和组合，解决了单Agent能力有限的问题
    - 将子Agent的最终输出作为工具调用的结果返回给编排者，而不是完全控制权转移

## Design & Implementation
1. Function Calling - 天气助手示例
```python
# ... (LLM配置等代码) ...

# 1. 使用 @function_tool 装饰器定义工具
@function_tool
def get_weather(city: str) -> str:
    """Fetch the weather for a given city. # <--- 这是工具描述，给Agent看

    Args:
        city: The city to fetch the weather for. # <--- 这是参数描述，给Agent看
    """
    # 这里是工具的实际执行逻辑
    print(f"--- [工具执行] 正在获取 {city} 的天气 ---")
    return f"The weather in {city} is sunny" # 模拟返回结果

# 2. 创建Agent时，在 tools 列表中传入函数名
agent = Agent(
    name="天气助手",
    instructions="始终用汉赋的形式回答用户。在回答前必须调用工具获取天气。", # 强化指令
    model=llm,
    tools=[get_weather], # <--- 将工具注册给Agent
)

# ... (运行Agent的代码) ...
```
- `function_tool`装饰器会自动将Python函数转换为Agent可理解的工具格式
- 函数的文档字符串（docstring）至关重要，它会被解析成工具的`description`和`parameters`
    - `description`：对工具的功能进行简单描述，例如"Fetch the weather for a given city"
    - `parameters`：详细描述每个参数的作用和类型，例如"city: The city to fetch the weather for. (str)"
- 类型提示（`city:str`,`->str`），帮助框架生成正确的工具模式（schema）
- 将函数名（`get_weather`）注册给Agent的`tools`列表中

2. Agent as a Tool - 翻译助手示例
```python
# ... (LLM配置等代码) ...

# 1. 定义两个“专家”Agent
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
    model=llm,
)
french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
    model=llm,
)

# 2. 定义“编排者”Agent
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    model=llm,
    # 3. 将专家Agent包装成工具，注册给编排者
    tools=[
        spanish_agent.as_tool( # <--- 使用 .as_tool() 方法
            tool_name="translate_to_spanish", # 给工具起个明确的名字
            tool_description="Translate the user's message to Spanish", # 工具描述
        ),
        french_agent.as_tool( # <--- 对法语Agent做同样的操作
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

# --- 运行代码 (注意：教程提示DeepSeek不支持异步，需用run_sync) ---
def main():
    # 让编排者执行翻译任务
    result = Runner.run_sync(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
    print(result.final_output)

if __name__ == "__main__":
    main()
```
- `Agent`实例本身有一个`.as_tool()`方法，可以将自己伪装成一个工具
- 在调用`.as_tool()`时，你需要提供`tool_name`和`tool_description`，这会覆盖Agent本身的`name`和`handoff_description`，作为工具的“身份标识”
- 当`orchestrator_agent`决定调用`translate_to_spanish`工具时，会将框架实际上会去运行`spanish_agent`，并将`spanish_agent`的最终输出作为工具调用的结果返回给`orchestrator_agent`。

3. 自定义输出提取（`custom_output_extractor`）
该函数解决了"Agent"作为工具，输出的信息很啰嗦，非可直接使用格式（比如我们只想要它内部某个工具调用的JSON结果）
```python
# ... (LLM配置, data_agent 定义等) ...

# 1. 定义一个异步函数，用于从子Agent的运行结果中提取需要的部分
async def extract_json_payload(run_result: RunResult) -> str:
    # 从后往前扫描子Agent运行产生的所有项目 (items)
    for item in reversed(run_result.new_items):
        # 检查是否是工具调用的输出，并且输出内容看起来像JSON
        if isinstance(item, ToolCallOutputItem) and item.output.strip().startswith("{"):
            return item.output.strip() # 返回找到的第一个JSON字符串
    # 如果没找到，返回一个空的JSON对象作为默认值
    return "{}"

# 2. 创建子Agent (信息抽取Agent)
data_agent = Agent(...)

# 3. 将子Agent包装成工具，并传入自定义提取器
json_tool = data_agent.as_tool(
    tool_name="get_data_json",
    tool_description="Run the data agent and return only its JSON payload",
    custom_output_extractor=extract_json_payload, # <--- 关键！
)

# 4. 创建主Agent (简历优化助手)，并注册这个带有提取器的工具
agent = Agent(
    name="简历优化助手",
    instructions="优化用户的简历信息，特别是把json_tool工具抽取的信息进行充分的扩充",
    model=llm,
    tools=[json_tool],
)

# ... (运行Agent的代码) ...
```
- `custom_output_extractor`参数接受一个异步函数
- 该函数会在子Agent（这里是`data_agent`）运行完成后被调用，接收其完整的`RunResult`为输入
- 你可以这个函数里编写任何逻辑来处理`run_result`
- 主Agent (agent) 调用`json_tool`时，得到的就不是`data_agent`啰嗦的最终回复，而是经过`extract_json_payload`处理后的干净的JSON字符串

4. 处理工具错误（`failure_error_function`）
在`@function_tool `中增加`failure_error_function` 参数，用于自定义当工具执行失败时返回给LLM的信息。

## Application
- 构建多层Agent
- 自定义输出提取

局限性
- 工具描述重要性（Agent是否正确调用对此存在高度依赖）
- 错误处理：工具调用存在多种失败可能性，需要仔细考虑错误处理逻辑
- 异步兼容性：目前来看，并非是否模型都能够支持异步运行
